In [1]:
import os
import sys
import random
import json
from itertools import product

import numpy as np
import torch
import torch.nn as nn

from collections import defaultdict

from torch.utils import data
from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.augmentation import NoiseHistories

sys.path.append(os.path.abspath("../src"))



In [2]:
# edge_types = list(product(AgentType, repeat=2))
# # edge_types
# for edge_type in edge_types:
#     print(edge_type[0].name)

In [3]:
arg_device = "cpu"
arg_seed = None

if not torch.cuda.is_available() or arg_device == 'cpu':
    arg_device = torch.device('cpu')
else:
    if torch.cuda.device_count() == 1:
        # If you have CUDA_VISIBLE_DEVICES set, which you should,
        # then this will prevent leftover flag arguments from
        # messing with the device allocation.
        arg_device = 'cuda:0'

    arg_device = torch.device(arg_device)

if arg_device is None:
    arg_device = 'cpu'

if arg_seed is not None:
    random.seed(arg_seed)
    np.random.seed(arg_seed)
    torch.manual_seed(arg_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(arg_seed)

In [4]:
log_dir = '/home/abbas/Projects/trajectron/adaptive-trajectron-plus-plus/experiments/pedestrians/kf_models'
model_dir = os.path.join(log_dir, 'eth_1mode_base_tpp-03_Dec_2024_13_01_59')

# Load hyperparameters from json
conf = 'config.json'
config_file = os.path.join(model_dir, conf)
if not os.path.exists(config_file):
    raise ValueError('Config json not found!')
with open(config_file, 'r') as conf_json:
    hyperparams = json.load(conf_json)

In [5]:
desired_data=[
    "eupeds_eth-train",
]
data_dirs = {
    "eupeds_eth": "~/Projects/trajectron/datasets/eth_ucy_peds",
}

attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

In [6]:
batch_size = 4

dataset = UnifiedDataset(
    desired_data=desired_data,
    history_sec=(0.1, hyperparams["history_sec"]),
    future_sec=(0.1, hyperparams["prediction_sec"]),
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
)

dataloader = data.DataLoader(
    dataset,
    collate_fn=dataset.get_collate_fn(pad_format="right"),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=batch_size,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

batch: AgentBatch = next(iter(dataloader))

Loading data for matched scene tags: ['train-zurich-eupeds_eth', 'train-eupeds_eth-cyprus']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 15592.21it/s]

1 scenes in the scene index.



Structuring Agent Data Index: 100%|██████████| 1/1 [00:00<00:00, 790.48it/s]


In [ ]:
model = nn.MultiheadAttention( #TODO: update attention module
    embed_dim=hp["enc_rnn_dim_history"],
    num_heads=1,
    kdim=hp["enc_rnn_dim_edge"],
    vdim=hp["enc_rnn_dim_edge"],
    batch_first=False,
)

In [18]:
max_norm = 0.0
min_norm = float('inf')
for batch in dataloader:
    # print(f"Num Neigh: {batch.neigh_hist.shape[1]}")
    for neigh_idx in range(batch.neigh_hist.shape[1]):
        # print(f"Timestep: {batch.scene_ts.item()}, Neighbour Index: {neigh_idx}")
        neigh_hist = batch.neigh_hist[0, neigh_idx, :, :2]
        valid_rows = ~torch.isnan(neigh_hist).all(dim=1)
        if valid_rows.any():
            filtered_neigh = neigh_hist[valid_rows]
            # print(f"Filtered Neighbour Histories: {filtered_neigh[-1].shape}")
            norm = torch.norm(filtered_neigh[-1])
            if norm.item() > max_norm:
                max_norm = norm.item()
            if norm.item() < min_norm:
                min_norm = norm.item()
print(f"Max norm: {max_norm}")
print(f"Min norm: {min_norm}")

Max norm: 4.998409748077393
Min norm: 0.4401136338710785
